In [1]:
lr = 1e-4
bs = 32
val_split = 0.2
num_epochs = 10
num_classes = 2
MOMENTUM = 0.9
epochs = 10

In [2]:
import gc
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils import data
import numpy as np
import torchvision
from  numpy import exp, absolute, asarray
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import time, os, json, random, gc, copy, math, multiprocessing
from sklearn import svm
import sklearn.model_selection as model_selection
from sklearn.metrics import accuracy_score,f1_score,precision_score ,recall_score 
from PIL import Image
from pathlib import Path

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
data_dir = str(Path().resolve().parent.parent.parent) + "/Datasets/Independent Datasets/E.Coli/sub/"

In [4]:
ho_transforms = transforms.Compose([transforms.ToTensor()])
ho_data = datasets.ImageFolder(data_dir, transform=ho_transforms)
holoader = torch.utils.data.DataLoader(ho_data, batch_size=bs)
print(ho_data.class_to_idx)

{'pos_sub': 0}


In [5]:
model = torch.load(str(Path().resolve().parent.parent) + "/Benchmark/S.Cervisiae/" + "finalmodel-full.pth")

In [6]:
print(model.classifier.in_features)

1920


In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)

In [8]:
ho_loss = 0
ho_accuracy = 0
res = []
orig_labels = []
model.eval()
with torch.no_grad():
    for inputs, labels in holoader:
        inputs = inputs.to(device)
        #labels = labels.type(torch.LongTensor) # <---- Here (casting)
        labels = labels.to(device)
        logps = model.forward(inputs)
        batch_loss = criterion(logps, labels)
        ho_loss += batch_loss.item()

        ps = torch.exp(logps)
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)
        res.extend(top_class)
        orig_labels.extend(labels)
        ho_accuracy += torch.mean(equals.type(torch.FloatTensor)).item()                
print(f"Test loss: {ho_loss/len(holoader):.3f}.. "
      f"Test accuracy: {ho_accuracy/len(holoader):.3f}")

Test loss: 5.322.. Test accuracy: 0.096


In [9]:
import pickle
import json

f = open(str(Path().resolve().parent.parent.parent) + "/Datasets/Independent Datasets/E.Coli/pimage_dict.json")
p_data = json.load(f)
f.close()

In [10]:
fin_op = []
fin_ip = []
for img in p_data:
    if(p_data[img]):
        outputs = [res[i - 1].item() for i in p_data[img]]
        fin_op.append(1 if np.mean(outputs)>=0.5 else 0)
        fin_ip.append(1)

In [11]:
print(len(fin_op), len(fin_ip))

4049 4049


In [12]:
correct = (np.array(fin_op) == np.array(fin_ip))
accuracy = correct.sum() / correct.size
print(accuracy)

0.9972832798221783
